In [1]:
import os
import re
from nltk.tokenize import TreebankWordTokenizer
import pandas as pd

In [20]:
ready_df = pd.DataFrame()

In [3]:
def preprocess(text):

    tokenized_text = TreebankWordTokenizer().tokenize(text)
    
    wordsFiltered = []

    for w in tokenized_text:
        for wf in re.findall(r'(?u)(\w+)', w):
            #for wfreal in re.findall(r'(\D+)', wf):
            wordsFiltered.append(''.join(wf))
    
    return wordsFiltered

In [21]:
def get_context(file_path):
    
    with open(file_path) as fl:
        try:
            text = fl.read()
        except:
            print ("Error: " + file_path )
    
    reg_term = re.compile(r'&(.*?)!&')
    indexes = []
    for i in reg_term.finditer(text):
        indexes.append((i.start(), i.end()))

    left_context = []
    right_context = []

    for ind in indexes:
        left_part = text[:ind[0]]
        right_part = text[ind[1]+1:]
        clear_left = preprocess(left_part)
        clear_right = preprocess(right_part)
    
    
        #left context
        if len(clear_left) > 5:
            left_context.append(clear_left[-5:])
        else:
            left_context.append(clear_left)
    
        #right context
        if len(clear_right) > 5:
            right_context.append(clear_right[:5])
        else:
            right_context.append(clear_right)
            
    
    terms = reg_term.findall(text)
    
    global ready_df
    
    for i in range(len(terms)):
        lft = left_context[i]
        term = terms[i]
        rgt = right_context[i]
        lc = ' '.join(lft)
        rc = ' '.join(rgt)
        df = pd.DataFrame([[lc, term, rc]],
                      columns=['left_context', 'term', 'right_context'])
        ready_df = ready_df.append(df)


In [22]:
start_path = './terms_markup/Marked/'

for path, dirs, files in os.walk(start_path):

    for fname in files:
        if not fname.startswith('.'):
            get_context(start_path+fname)


In [23]:
len(ready_df)

1133

In [27]:
ready_df.tail(500)

,left_context,term,right_context
0,экологии составляет своеобразную гармонию с,правовым нигилизмом,и политическим цинизмом
0,тромб закупоривает сосуд поэтому с,системой свертывания крови,шутки плохи Что у системы
0,крови шутки плохи Что у,системы свертывания крови,общего с Войной и миром
0,выпуске рубрики Чем нас лечат,Ишемическая болезнь сердца,и инсульт две самые распространенные
0,Ксарелто который назначают для профилактики,тромбоза,и инсульта В России препарат
0,внутривенно Действующее вещество Ксарелто назы...,ривароксабан,и содержится в таблетке в
0,в количестве 15 мг Структура,ривароксабана,Brenton Wikimedia Commons Это вещество
0,в продажу Согласно исследованиям действие,ривароксабана,основано на том что молекула
0,S1 и S4 Иллюстрация действия,ривароксабана,Nature Reviews Drug Discovery Попробуем
0,имена у героев очень похожи,протромбин,и тромбин фибриноген и фибрин


In [25]:
ready_df.to_csv('terms_context.csv', sep='\t')